In [35]:
!pip install tensorflow opencv-python-headless face_recognition

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import face_recognition
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
directories = [
    "/content/drive/MyDrive/Reconhecimento_Facial/cadastrados",
    "/content/drive/MyDrive/Reconhecimento_Facial/frequência"
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [37]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import IPython.display as display
from PIL import Image
import io

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to match the aspect ratio of the video.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks().forEach(track => track.stop());
        div.remove();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        return dataUrl;
    }
    ''')
    display.display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [92]:
model_save_path = '/content/drive/MyDrive/Reconhecimento_Facial/saved_model_3'
model = load_model(model_save_path)

In [ ]:
import os
import time
import shutil
from datetime import datetime
import numpy as np
from PIL import Image
import face_recognition

cadastrados_path = "/content/drive/MyDrive/Reconhecimento_Facial/cadastrados"
frequency_path = "/content/drive/MyDrive/Reconhecimento_Facial/frequência"

def add_new_person(user_id, user_name):
    if not user_id or not user_name:
        print("Erro: ID e nome do usuário são obrigatórios.")
        return

    image_path = take_photo(filename='new_user_image.jpg')
    if is_face_registered(image_path=image_path):
        os.remove(image_path)
        print(f"Erro: Rosto já está cadastrado.")
        return

    final_image_path = os.path.join(cadastrados_path, f"{user_id}_{user_name}.jpg")
    shutil.move(image_path, final_image_path)
    print(f"Imagem capturada e salva em {final_image_path}")

    log_file = os.path.join(frequency_path, f"{user_id}.txt")
    with open(log_file, "w") as f:
        f.write(f"Usuário: {user_name}\nEntrada: \nSaída: \n")
    print(f"Arquivo de log criado para {user_name} em {log_file}")

def add_new_person_from_image(user_id, user_name, image_path):
    if not os.path.isfile(image_path):
        print(f"Erro: Caminho da imagem {image_path} não é válido.")
        return

    if is_face_registered(image_path=image_path):
        print(f"Erro: Rosto já está cadastrado.")
        return

    final_image_path = os.path.join(cadastrados_path, f"{user_id}_{user_name}.jpg")
    shutil.move(image_path, final_image_path)
    print(f"Imagem salva em {final_image_path}")

    log_file = os.path.join(frequency_path, f"{user_id}.txt")
    with open(log_file, "w") as f:
        f.write(f"Usuário: {user_name}\nEntrada: \nSaída: \n")
    print(f"Arquivo de log criado para {user_name} em {log_file}")

def is_user_id_registered(user_id):
    return any(filename.startswith(f"{user_id}_") for filename in os.listdir(cadastrados_path))

def is_user_name_registered(user_name):
    return any(filename.split('_')[1].replace('.jpg', '') == user_name for filename in os.listdir(cadastrados_path))

def is_face_registered(image_path=None):
    if image_path:
        new_image = face_recognition.load_image_file(image_path)
    else:
        new_image_path = take_photo(filename='new_user_image.jpg')
        new_image = face_recognition.load_image_file(new_image_path)

    new_encodings = face_recognition.face_encodings(new_image)
    if not new_encodings:
        print("Erro: Nenhum rosto detectado na nova imagem.")
        return True

    new_encoding = new_encodings[0]

    for filename in os.listdir(cadastrados_path):
        if filename.endswith(".jpg"):
            registered_image_path = os.path.join(cadastrados_path, filename)
            registered_image = face_recognition.load_image_file(registered_image_path)
            registered_encodings = face_recognition.face_encodings(registered_image)

            if registered_encodings:
                registered_encoding = registered_encodings[0]
                distance = face_recognition.face_distance([registered_encoding], new_encoding)
                if distance <= 0.6:
                    return True
    return False

def log_attendance(user_id, user_name):
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")

    log_file = os.path.join(frequency_path, f"{user_id}.txt")
    if os.path.exists(log_file):
        with open(log_file, "r+") as f:
            lines = f.readlines()
            if 'Entrada: \n' in lines:
                lines[1] = f"Entrada: {current_time}\n"
                lines[2] = f"Saída: {current_time}\n"
            else:
                lines[2] = f"Saída: {current_time}\n"
            f.seek(0)
            f.writelines(lines)
    else:
        with open(log_file, "w") as f:
            f.write(f"Usuário: {user_name}\nEntrada: {current_time}\nSaída: {current_time}\n")

def recognize_faces_and_log():
    captured_image_path = take_photo(filename='captured_image.jpg')
    captured_image = face_recognition.load_image_file(captured_image_path)
    captured_encodings = face_recognition.face_encodings(captured_image)

    if not captured_encodings:
        print("Nenhuma face encontrada na imagem capturada.")
        os.remove(captured_image_path)
        return

    captured_encoding = captured_encodings[0]

    best_match_distance = float('inf')
    best_match_user_id = None
    best_match_user_name = None

    for filename in os.listdir(cadastrados_path):
        if filename.endswith(".jpg"):
            user_id, user_name = filename.split('_', 1)
            user_name = user_name.replace('.jpg', '')
            registered_image_path = os.path.join(cadastrados_path, filename)
            registered_image = face_recognition.load_image_file(registered_image_path)
            registered_encodings = face_recognition.face_encodings(registered_image)

            if registered_encodings:
                registered_encoding = registered_encodings[0]
                distance = face_recognition.face_distance([registered_encoding], captured_encoding)

                if distance < best_match_distance:
                    best_match_distance = distance
                    best_match_user_id = user_id
                    best_match_user_name = user_name

    if best_match_user_id is not None and best_match_distance <= 0.6:
        log_attendance(best_match_user_id, best_match_user_name)
        print(f"Presença confirmada para {best_match_user_name}")
    else:
        print("Rosto não reconhecido")

    os.remove(captured_image_path)
    time.sleep(2)

def clear_frequency_folder():
    for filename in os.listdir(frequency_path):
        file_path = os.path.join(frequency_path, filename)
        if os.path.isfile(file_path):
            os.unlink(file_path)
    print("Pasta de frequência esvaziada para um novo dia.")

def menu():
    while True:
        print("Sistema de Frequência por Reconhecimento Facial")
        print("1. Adicionar nova pessoa (captura de câmera)")
        print("2. Adicionar nova pessoa (a partir de imagem)")
        print("3. Verificar presença")
        print("4. Limpar frequência para um novo dia")
        print("5. Sair")

        choice = input("Escolha uma opção: ")

        if choice == "1":
            user_id = input("Digite o ID da pessoa: ")
            user_name = input("Digite o nome da pessoa: ")
            add_new_person(user_id, user_name)
        elif choice == "2":
            user_id = input("Digite o ID da pessoa: ")
            user_name = input("Digite o nome da pessoa: ")
            image_path = input("Digite o caminho da imagem: ")
            add_new_person_from_image(user_id, user_name, image_path)
        elif choice == "3":
            recognize_faces_and_log()
        elif choice == "4":
            clear_frequency_folder()
        elif choice == "5":
            print("Saindo do sistema...")
            break
        else:
            print("Opção inválida. Tente novamente.")

menu()
